In [1]:
import nlp
import numpy as np
from transformers import BertTokenizer

def extract_sentences():
    dataset = nlp.load_dataset('glue', "sst2")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    results = []
    for sent in dataset['train']['sentence']:
        results.append(tokenizer.tokenize(sent))
    return results

In [2]:
sentences = extract_sentences()

In [3]:
sentences = sentences * 4
len(sentences)

269396

In [4]:
from copy import deepcopy

In [5]:
def add_special_tokens(sentence):
    sentence.insert(0, "[CLS]")
    sentence.append("[SEP]")

In [6]:
%%timeit
tmp = deepcopy(sentences)

356 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
tmp = deepcopy(sentences)
for sentence in tmp:
    add_special_tokens(sentence)

397 ms ± 8.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
tmp = deepcopy(sentences)

In [9]:
%%time
for sentence in tmp:
    add_special_tokens(sentence)

CPU times: user 53.2 ms, sys: 28 µs, total: 53.2 ms
Wall time: 52.6 ms


In [10]:
def is_first_piece(tokens):
    return [not token.startswith("##") for token in tokens]

In [11]:
is_first_piece(["1", "w", "##e"])

[True, True, False]

In [12]:
%%time
tmp = deepcopy(sentences)

CPU times: user 505 ms, sys: 27 µs, total: 506 ms
Wall time: 505 ms


In [13]:
%%timeit
results = [is_first_piece(sent) for sent in tmp]

331 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
results = [is_first_piece(sent) for sent in tmp]
results[0]

[True, True, True, False, True, True, True, True]